In [1]:
from mycelia.shared.config import MinerConfig, ValidatorConfig
from mycelia.shared.cycle import *
from mycelia.shared.chain import * 
from mycelia.miner.train import * 
from mycelia.miner.model_io import *
from mycelia.shared.checkpoint import *
import bittensor as bt 

PyTorch version 2.9.1 available.


/home/isabella/crucible/subnet-MoE/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
subtensor = bt.Subtensor('test')
path = "/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/config.yaml"
miner_config = MinerConfig.from_path(path)

path = "/home/isabella/crucible/subnet-MoE/checkpoints/validator/validator/hk1/foundation/config.yaml"
vali_config = ValidatorConfig.from_path(path)

rank = 2
wallet, subtensor = setup_chain_worker(miner_config)
device = torch.device(f"cuda:{rank}" if torch.cuda.is_available() else "cpu")
tokenizer = get_base_tokenizer(miner_config)
expert_manager = ExpertManager(miner_config)

2025-12-16 06:50:05 [info     ] load_expert_group_assignment - folder [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_dummy'),)
2025-12-16 06:50:05 [info     ] load_expert_group_assignment - folder [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_math'),)


## Commit 

In [11]:
commits = get_chain_commits(miner_config, subtensor)
validator_seeds = get_validator_seed_from_commit(miner_config, commits)
# miners = get_miners_from_commit(config, commits)
# return assign_miners_to_validators(validator_seeds, miners)  #

for c in commits:
    print(c)

print(validator_seeds)

(None, NeuronInfoLite(hotkey='5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH', coldkey='5EXGTzS8r36MrKuYGknxmyqsX4eModUaSWJKn1eXVqeoBJRr', uid=16, netuid=348, active=False, stake=τ0.000000000, stake_dict={'5EXGTzS8r36MrKuYGknxmyqsX4eModUaSWJKn1eXVqeoBJRr': τ0.000000000}, total_stake=τ0.000000000, rank=0.0, emission=0.0, incentive=0.0, consensus=0.0, trust=0.0, validator_trust=0.0, dividends=0.0, last_update=5750573, validator_permit=False, prometheus_info=PrometheusInfo(block=0, version=0, ip='0.0.0.0', port=0, ip_type=0), axon_info=AxonInfo( /ipv4/149.137.225.62:8001, 5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH, 5EXGTzS8r36MrKuYGknxmyqsX4eModUaSWJKn1eXVqeoBJRr, 10000000 ), pruning_score=0, is_null=False))
(ValidatorChainCommit(model_hash='xxx', model_version=94, expert_group=1, miner_seed=3024650, block=None), NeuronInfoLite(hotkey='5DUqbhkpvth1WzRogVU9NvGeV9bwX3wBHFWHMZk72WEhZjHF', coldkey='5EXGTzS8r36MrKuYGknxmyqsX4eModUaSWJKn1eXVqeoBJRr', uid=15, netuid=348, active=False, st

In [13]:
all_commitments = subtensor.get_all_commitments(netuid=miner_config.chain.netuid)

In [19]:
import json
ValidatorChainCommit.model_validate(json.loads(all_commitments['5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH']))

ValidatorChainCommit(model_hash=None, model_version=94, expert_group=0, miner_seed=62772, block=6047967)

In [22]:
for hotkey, commit in all_commitments.items():
    if hotkey != '5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH':
        continue

    print(hotkey)
    # try:
    status_dict = json.loads(commit)

    chain_commit = (
        ValidatorChainCommit.model_validate(status_dict)
        if "miner_seed" in status_dict
        else MinerChainCommit.model_validate(status_dict)
    )

5DoHdXfDYraqPzkLjrXGMZxvGXYdDYhuC8tGbQdb4zvz2LbH


ValidationError: 1 validation error for MinerChainCommit
block
  Field required [type=missing, input_value={'h': None, 'v': 94, 'e':...s': 62772, 'b': 6047967}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing